In [1]:
import datasets
import numpy as np
import evaluate
import torch
import torch.nn as nn
import torch.optim as optim
import nltk
import string
import json
import joblib
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report,accuracy_score
from collections import Counter

In [2]:
# 下載 nltk 資源（只需一次）
nltk.download('punkt')
nltk.download("punkt_tab")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fenne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Fenne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fenne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#有GPU用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#Load the dataset
dataset = datasets.load_from_disk("super-emotion")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

texts_train  = train_dataset["text"]
labels_train = train_dataset["labels"]

texts_test = test_dataset["text"]
labels_test = test_dataset["labels"]

### Data Preprocessing(torch)

In [5]:
# NLTK prerocessing
stop_words = set(stopwords.words("english"))
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha()]  # 移除標點、數字
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)
def preprocess_and_save(texts, output_file, batch_size=1000):
    with open(output_file, "w", encoding="utf-8") as f:
        for i in tqdm(range(0, len(texts), batch_size)):
            batch = texts[i:i + batch_size]
            cleaned_batch = [preprocess_text(t) for t in batch]
            for line in cleaned_batch:
                f.write(json.dumps(line) + "\n")
preprocess_and_save(texts_train, "texts_train_cleaned.jsonl")
preprocess_and_save(texts_test, "texts_test_cleaned.jsonl")
with open("labels_train.json", "w", encoding="utf-8") as f:
    json.dump(labels_train, f)
with open("labels_test.json", "w", encoding="utf-8") as f:
    json.dump(labels_test, f)
def load_cleaned_texts(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]
texts_cleaned_train = load_cleaned_texts("texts_train_cleaned.jsonl")
texts_cleaned_test = load_cleaned_texts("texts_test_cleaned.jsonl")
#RAM會爆炸!!
#texts_cleaned_train = [preprocess_text(t) for t in texts_train [:50000]]
#texts_cleaned_test = [preprocess_text(t) for t in texts_test [:50000]]
#labels_subset_train = labels_train[:50000]
#labels_subset_test = labels_test[:50000]

100%|██████████| 59/59 [00:04<00:00, 13.48it/s]


In [7]:
# 設定 vectorizer，直接使用 float32，減少轉型開銷
vectorizer = TfidfVectorizer(max_features=10000, dtype=np.float32)
vectorizer.fit(texts_cleaned_train)

# 定義記憶體優化版的批次轉換
def vectorize_in_batch(texts, batch_size=50):  # 可視情況調整 batch_size
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        # 直接產生 float32 稀疏矩陣，再轉成 dense
        X_batch = vectorizer.transform(batch).toarray()  # 已是 float32
        yield torch.tensor(X_batch)  # 生成 PyTorch tensor

# 使用範例 - 訓練資料
X_train_batches = []
for X_batch_tensor in vectorize_in_batch(texts_cleaned_train):
    X_train_batches.append(X_batch_tensor)

# 使用範例 - 測試資料
X_test_batches = []
for X_batch_tensor in vectorize_in_batch(texts_cleaned_test):
    X_test_batches.append(X_batch_tensor)

In [8]:
# Label multi-hot encoding
#mlb = MultiLabelBinarizer()
# Y = mlb.fit_transform(labels_subset_train)   # shape = (n_samples, n_emotions)
#y_train = mlb.fit_transform(labels_subset_train)
#y_test = mlb.transform(labels_subset_test)
#y_train = mlb.fit_transform(labels_train)   # shape = (n_samples, n_emotions)
#y_test = mlb.transform(labels_test)

# 分割訓練集與測試集
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#X_train = torch.tensor(X_train, dtype=torch.float32)
#X_test = torch.tensor(X_test, dtype=torch.float32)
#原本為多標籤，取第一個標籤作為分類
y_train_single = [labels[0] for labels in labels_train]
y_test_single = [labels[0] for labels in labels_test]
#轉成tensor
train_data = []
start_idx = 0
for batch in X_train_batches:
    current_batch_size = batch.shape[0]
    end_idx = start_idx + current_batch_size
    train_data.append((batch, torch.tensor(y_train_single[start_idx:end_idx], dtype=torch.long)))
    start_idx = end_idx
test_data = []
start_idx = 0
for batch in X_test_batches:
    current_batch_size = batch.shape[0]
    end_idx = start_idx + current_batch_size
    test_data.append((batch, torch.tensor(y_test_single[start_idx:end_idx], dtype=torch.long)))
    start_idx = end_idx
#y_train = torch.tensor(y_train_single, dtype=torch.long)
#y_test = torch.tensor(y_test_single, dtype=torch.long)

In [9]:
#建立模型
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.net=nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim//2, output_dim)
        )
    def forward(self, x):
        return self.net(x)

In [10]:

#設定類別的權重，因為資料集不平衡
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(7),
    y=y_train_single
)
class_weights = torch.tensor(class_weights, dtype=torch.float32)
#設定參數
input_dim = X_train_batches[0].shape[1]
hidden_dim = 256
output_dim = 7
#初始化模型
model = MLP(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=0.0001)
#訓練模型
epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    for X_batch, y_batch in train_data:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(y_batch.cpu().tolist())
    acc = accuracy_score(all_labels, all_preds)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_data):.4f}, Acc: {acc:.4f}")
torch.cuda.empty_cache()

Epoch [1/50], Loss: 0.8502, Acc: 0.7092
Epoch [2/50], Loss: 0.4910, Acc: 0.8024
Epoch [3/50], Loss: 0.4493, Acc: 0.8108
Epoch [4/50], Loss: 0.4229, Acc: 0.8173
Epoch [5/50], Loss: 0.4048, Acc: 0.8218
Epoch [6/50], Loss: 0.3903, Acc: 0.8256
Epoch [7/50], Loss: 0.3781, Acc: 0.8299
Epoch [8/50], Loss: 0.3675, Acc: 0.8334
Epoch [9/50], Loss: 0.3571, Acc: 0.8373
Epoch [10/50], Loss: 0.3476, Acc: 0.8410
Epoch [11/50], Loss: 0.3384, Acc: 0.8447
Epoch [12/50], Loss: 0.3299, Acc: 0.8485
Epoch [13/50], Loss: 0.3219, Acc: 0.8513
Epoch [14/50], Loss: 0.3148, Acc: 0.8543
Epoch [15/50], Loss: 0.3069, Acc: 0.8573
Epoch [16/50], Loss: 0.3002, Acc: 0.8600
Epoch [17/50], Loss: 0.2938, Acc: 0.8631
Epoch [18/50], Loss: 0.2870, Acc: 0.8656
Epoch [19/50], Loss: 0.2814, Acc: 0.8683
Epoch [20/50], Loss: 0.2765, Acc: 0.8705
Epoch [21/50], Loss: 0.2715, Acc: 0.8727
Epoch [22/50], Loss: 0.2660, Acc: 0.8746
Epoch [23/50], Loss: 0.2605, Acc: 0.8772
Epoch [24/50], Loss: 0.2576, Acc: 0.8780
Epoch [25/50], Loss: 0.25

In [11]:
model.eval()
total_loss = 0.0
all_preds = []
all_labels = []
with torch.no_grad():
    for X_batch, y_batch in test_data:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        all_preds.append(preds)
        all_labels.append(y_batch)
    avg_loss = total_loss / len(test_data)
    all_preds = torch.cat(all_preds).cpu().numpy()
    all_labels = torch.cat(all_labels).cpu().numpy()
    acc = accuracy_score(all_labels, all_preds)
    print(f"Test Loss: {avg_loss:.4f}, Test Acc: {acc:.4f}")

Test Loss: 0.8828, Test Acc: 0.7678


In [12]:
torch.save(model.state_dict(), "mlp_weights.pth")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

### Data preprocessing(sklearning)

In [16]:
# NLTK prerocessing
stop_words = set(stopwords.words("english"))
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha()]  # 移除標點、數字
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

texts_cleaned_train = [preprocess_text(t) for t in texts_train]
texts_cleaned_test = [preprocess_text(t) for t in texts_test]

In [ ]:
# TF-IDF 向量化
vectorizer = TfidfVectorizer(max_features=10000)
# X = vectorizer.fit_transform(texts_cleaned_train)
X_train = vectorizer.fit_transform(texts_cleaned_train)
X_test = vectorizer.transform(texts_cleaned_test)

In [ ]:
# Label multi-hot encoding
mlb = MultiLabelBinarizer()
# Y = mlb.fit_transform(labels_train)   
y_train = mlb.fit_transform(labels_train) # shape = (n_samples, n_emotions)
y_test = mlb.transform(labels_test)

# 分割訓練集與測試集
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
sklearn_model=LogisticRegression(max_iter=1000)
clf= OneVsRestClassifier(sklearn_model)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.fit(X_train, y_train)
print(classification_report(y_test, y_pred, target_names=mlb.classes_))